# Conv_BoDiEs 학습
Dataset: https://skeletex.xyz/portfolio/datasets
(Estimation of Anthopometric human body measurements)

## 1. 라이브러리 import, 데이터 경로 지정정

In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms as transforms
import os
import zipfile
import torch

# 구글 드라이브 마운트 - Colab 사용할 경우
from google.colab import drive
drive.mount('/content/drive')

# 훈련을 위한 CPU 또는 GPU 장치 가져오기
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device} 장치를 사용 중입니다")
if torch.cuda.is_available():
    print(f'장치 이름: {torch.cuda.get_device_name(0)}')

# 이미지 ZIP 파일 압축 해제 / 드라이버에서의 위치를 지정 /Colab을 사용하지 않을 경우 local에서의 데이터 경로 지정정
zip_file_path = '/content/drive/MyDrive/Conv_BoDiEs/dataset/imgs_female.zip'  # 압축 해제할 파일 경로
extract_dir = '/content/images_female'  # 압축 해제할 디렉토리

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print('Extraction completed')


Mounted at /content/drive
Using cuda device
Device name: Tesla T4
extraction completed


## 2. 데이터 로더 클래스 생성 및 학습/시험 데이터 분리리

In [4]:
# 데이터셋 클래스 정의 - 이미지와 라벨을 하나로 묶어서 Dataloader 생성성
class BodyMeasurementsDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file, header=None)  # header=None 옵션을 사용해 첫 번째 행도 데이터로 입력
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # 인덱스 기반으로 이미지 이름 생성
        img_name = os.path.join(self.img_dir, f"{idx:06d}.png")
        image = Image.open(img_name).convert('L')
        label = torch.tensor(self.data_frame.iloc[idx].values, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

# 데이터 전처리 및 변환 설정
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # 이미지를 200x200 크기로 리사이즈
    transforms.ToTensor()           # 이미지를 Tensor로 변환
])

# CSV 파일 경로와 이미지 디렉토리 설정
csv_file = '/content/drive/MyDrive/Conv_BoDiEs/dataset/Annotations/bodymeasurements_f.csv'  # CSV 파일 경로
img_dir = '/content/images_female/imgs_female'   # 압축 해제된 이미지 디렉토리 경로

# 전체 데이터셋을 로드
dataset = BodyMeasurementsDataset(csv_file=csv_file, img_dir=img_dir, transform=transform)

# 학습 데이터와 시험 데이터로 나눔
train_size = int(0.8 * len(dataset))  # 학습 데이터 비율 (80%)
test_size = len(dataset) - train_size  # 시험 데이터 비율 (20%)

# train_size는 40000, test_size는 10000이 됨 
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 학습용 DataLoader와 시험용 DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Google Drive에 저장할 디렉토리 생성 / Colab에서 실행하지 않을 경우 drive_path를 local 폴더로 지정 
drive_path = '/content/drive/My Drive/dataloader'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# DataLoader의 데이터를 Google Drive에 저장 / Colab에서 실행하지 않을 경우 drive_path를 local 폴더로 지정 
train_data_path = os.path.join(drive_path, 'train_data_fe_grayscale.pth')
test_data_path = os.path.join(drive_path, 'test_data_fe_grayscale.pth')

torch.save([data for data in train_loader], train_data_path)
torch.save([data for data in test_loader], test_data_path)

# 데이터셋 크기 출력
print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Training dataset size: 40000
Test dataset size: 10000


## 3. "Conv_BoDiES" 모델 아키텍처 생성성

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Conv_BoDiEs(nn.Module):
    def __init__(self):
        super(Conv_BoDiEs, self).__init__()

        # 첫 번째 컨볼루션 레이어
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 두 번째 컨볼루션 레이어
        self.conv2 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 세 번째 컨볼루션 레이어
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 네 번째 컨볼루션 레이어
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.flatten = nn.Flatten()

        # 평탄화된 크기 계산 (컨볼루션 후 크기)
        flattened_size = 32 * 12 * 12

        # 첫 번째 완전 연결 레이어
        self.fc1 = nn.Sequential(
            nn.Linear(flattened_size, 128),
            nn.ReLU()
        )

        # 두 번째 완전 연결 레이어
        self.fc2 = nn.Linear(128, 16)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# 모델 생성 및 사용
model = Conv_BoDiEs().to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0004)

In [ ]:
# 모델, 손실 함수, 옵티마이저 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 100
patience = 10  # 조기 종료를 위한 인내심 설정
best_loss = float('inf')
counter = 0


for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    epoch_loss = 0.0
    for images, labels in train_loader:  # 학습용 DataLoader에서 배치 데이터를 가져옴
        images, labels = images.to(device), labels.to(device)  # 데이터를 현재 장치(CPU/GPU)로 이동
        optimizer.zero_grad()  # 옵티마이저의 기울기를 초기화

        outputs = model(images)  # 모델을 통해 예측값을 계산
        loss = criterion(outputs, labels)  # 계산된 예측값과 실제 레이블로 손실 계산

        loss.backward()  # 손실에 대한 기울기를 계산(역전파)
        optimizer.step()  # 계산된 기울기를 사용하여 모델의 가중치를 업데이트

        epoch_loss += loss.item()  # 에폭 손실에 배치 손실을 더함

    epoch_loss /= len(train_loader)  # 평균 에폭 손실 계산
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # 조기 종료 로직
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        counter = 0
        # 최적의 모델 상태를 저장 / Colab 아닐시 local 경로로로 변경
        best_model_save_path = '/content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth'
        torch.save(model.state_dict(), best_model_save_path)
        print(f'최적 모델 저장: {best_model_save_path}')
    else:
        counter += 1
        if counter >= patience:  # 설정된 인내심을 초과하면 학습 중단
            print(f'조기 종료, Epoch: {epoch+1}')
            break

# 학습 완료 후 최종 모델 저장 / Colab 아닐시 local 경로로로 변경
final_model_save_path = '/content/drive/MyDrive/Conv_BoDiEs/Final_Conv_BoDiEs_model_fe_grayscale.pth'
torch.save(model.state_dict(), final_model_save_path)
print(f'최종 모델 저장: {final_model_save_path}')

Epoch [1/100], Loss: 2.6073
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [2/100], Loss: 1.5417
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [3/100], Loss: 1.3527
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [4/100], Loss: 1.2150
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [5/100], Loss: 1.1248
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [6/100], Loss: 1.0291
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [7/100], Loss: 0.9664
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [8/100], Loss: 0.9040
Best model saved to /content/drive/MyDrive/Conv_BoDiEs/Best_Conv_BoDiEs_model_fe_grayscale.pth
Epoch [9/100], L